In [ ]:
from mantid.simpleapi import *
from mantid.api import MatrixWorkspace
from mantid.api import AnalysisDataService as ADS
from mantid.api import *
from mantid.kernel import *
from sansdata import *

## 1. Create `Workspace` from data and instrument definition

In [ ]:
sans_file = SansData("data/memb_BS_Q1_6_0Ang.mpa")
sans_file.plot_2d()

In [ ]:
nBins = 1

delta_L_over_L0 = 0.1
bin_lower = sans_file.L0 * (1.0 - delta_L_over_L0/2)
bin_upper = sans_file.L0 * (1.0 + delta_L_over_L0/2)

detectors = 1024 * 1024

# Use the same bin for each
x = np.tile([bin_lower,bin_upper],detectors)
y = sans_file.raw_intensity

ws = CreateWorkspace(OutputWorkspace = "raw_2D_counts",UnitX="Wavelength",DataX=x, DataY=y, NSpec=detectors)
mon = LoadInstrument(ws, FileName="RIDSANS_Definition.xml", RewriteSpectraMap=True)
MoveInstrumentComponent(ws, ComponentName="sample-position", X=0.0, Y=0.0, Z=-sans_file.d, RelativePosition = False)



## 2. Verify basic Workspace properties

In [ ]:
# Print some basic instrument and workspace information
inst = ws.getInstrument()
si = ws.spectrumInfo()
di = ws.detectorInfo()
ci = ws.componentInfo()
print("Workspace {0} has instrument: {1}".format(ws.name(), inst.getName()))
print("Instrument {0} has {1} components, including {2} monitors and {3} detectors".format(inst.getName(), ci.size(), len(mon), di.size()))


unit = ws.getAxis(0).getUnit().unitID()
unit2 = ws.getAxis(1).getUnit().unitID()
print(unit)
print(unit2)

# Get the sample position
instrument = ws.getInstrument()

sample = instrument.getSample()
sample_position_1 = sample.getPos()

# sample_position_2 = si.samplePosition()

# print(spectrum_info, detector_info)
print("Sample Position:", sample_position_1)
# print("Sample Position:", sample_position_2)

num_spec = ws.getNumberHistograms()
num_bins = ws.blocksize()
print("Detector spectrum blocksize:", num_bins)
print("Number of spectra:", num_spec)
for i in range(num_spec):
    # Check that all spectra have a matching detector
    assert(si.hasDetectors(i))
    # Assert that at this point no spectrum is masked
    assert(not si.isMasked(i))
    # print(si.position(i))

The following code probes the histograms of a cross section of the detectors, looking for detectors with a particularly high count. It also verifies that the bin (X-axis) is correct.

In [ ]:
print("number of histograms = {0}".format(ws.getNumberHistograms()))
# ws.getSpectrum(3)
for i in range(0,ws.getNumberHistograms(),100):
    sum_counts = ws.readY(i)[0]

    # Display spectrum number against sum_counts
    if sum_counts >= 100.0:
        print("{0} {1} {2}".format(ws.getSpectrum(i).getSpectrumNo(), sum_counts,ws.readX(i)))

## 2.1. Calculate $Q_{max}$ and run `Qxy` algorithm

In [ ]:
max_det_x = 0.140662
max_det_y = 0.140662
ds_dist = sans_file.d
r = np.sqrt(2) * max_det_x
Q_max = 4 * np.pi / sans_file.L0 * np.sin(np.arctan(r/(2 * ds_dist)) / 2)
Q_max # AA-1

In [ ]:
delta_Q = 0.0001
max_QXY = 0.01
N_Q_bins = int(np.ceil(2*max_QXY/delta_Q))

# ws_scaled = ws / 1000
Qxy(ws,SolidAngleWeighting=False,MaxQxy=max_QXY, DeltaQ=delta_Q,OutputWorkspace="new")
reduced_ws = ADS.retrieve('new')
# dir(reduced_ws)

Q_array = np.zeros((N_Q_bins, N_Q_bins))
# Q_axis = np.zeros((N_Q_bins))
Q_axis = np.array(reduced_ws.dataX(0))
for i in range(N_Q_bins):
    Q_array[i] = reduced_ws.dataY(i)
Q_axis, Q_array

In [ ]:
from matplotlib.colors import LogNorm
# print(Q_array)
extent = [Q_axis[0], Q_axis[-1], Q_axis[0], Q_axis[-1]]
plt.imshow(Q_array, cmap='viridis', extent=extent, norm=LogNorm(),aspect='auto')  # cmap defines the color map (optional)
plt.colorbar()  # Add a colorbar to show the color scale
plt.xlabel(r'$Q_x$ [Å$^{-1}$]')
plt.ylabel(r'$Q_y$ [Å$^{-1}$]')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

In [ ]:
plt.imshow(Q_array, cmap='viridis')  # cmap defines the color map (optional)
plt.colorbar()  # Add a colorbar to show the color scale
plt.show()